In [ ]:
from google.cloud.bigquery.client import Client
from google.cloud import bigquery
from google.cloud import storage
from mlutils import dataset,connector
from datetime import datetime,timedelta
import pandas as pd
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import subprocess
import sys
import os
from subprocess import run as run
from subprocess import PIPE as PIPE

HOME = '/home/jupyter/'

upd_post_date  =  (str(run([f' ''curl --proxy1.0 \'http://sysproxy.wal-mart.com:8080\' -L -H \'Token: mde170cede5311120e6b6694d736356c1\' \'https://www.irs.gov/charities-non-profits/exempt-organizations-business-master-file-extract-eo-bmf\' |grep \'Updated data posting date\'  |awk -F\'g>\' \'{print $2}\' |awk -F\'</\' \'{print $1}\'  '], stdout=PIPE, stderr=PIPE, shell=True).stdout).replace("b'", "")).replace("\\n'", "")                      

post_date  = upd_post_date.split('/')[2]+upd_post_date.split('/')[0]+upd_post_date.split('/')[1]
list_date  = upd_post_date.split('/')[2]+'-'+upd_post_date.split('/')[0]+'-'+upd_post_date.split('/')[1]

seed  =  !chmod 600 /home/jupyter/.ssh/id_rsa; ssh b0l04yh@inkiru-ds-prod1.us-central1.us.walmart.net adquery user -a svcgecad |grep gdap |awk -F'/' '{print $NF}'
seed  =  seed[0]


run([f'cat {HOME}/srv.enc | ssh b0l04yh@inkiru-ds-prod1.us-central1.us.walmart.net openssl base64 -d -k {seed} > {HOME}srv.plain'], shell=True, encoding='utf-8')
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = HOME+'/srv.plain'
bq_cli = Client()
bqclient  = bigquery.Client()


ink_ds_list = list(bq_cli.list_datasets())
project     = bq_cli.project

run([f'rm {HOME}/srv.plain'], shell=True, encoding='utf-8')

eo_bmf_sch = bqclient.schema_from_json(f'{HOME}eo_bmf_schema.json')

eo_bmf_fig = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    schema=eo_bmf_sch,
    skip_leading_rows=1,
    ignore_unknown_values=True
)

with open(f'{HOME}eo_bmf_report.csv', 'w') as msg_1:
    msg_1.writelines('\n\n\tReport of Monthly IRS Exempt Org Bus file extract\n\n\r[ EO_BMF_EXTRACT.nonprofit_full_list '+upd_post_date+' ]')
    msg_1.writelines('\n\n\n')

run([f'touch {HOME}eo_bmf_region.csv; rm {HOME}/eo_bmf_region.csv'], shell=True, encoding='utf-8')

eo_bmf_dict=dict()

for region in [ 'Region 1:', 'Region 2:', 'Region 3:', 'Region 4:']:
    eo_file = (str(run([f' ''curl --proxy1.0 \'http://sysproxy.wal-mart.com:8080\' -L -H \'Token: mde170cede5311120e6b6694d736356c1\' \'https://www.irs.gov/charities-non-profits/exempt-organizations-business-master-file-extract-eo-bmf\' |grep "'+region+'".*pup_linkit_media  |awk -F\'g>\' \'{print $1}\' |awk -F\'</\' \'{print $1}\'  |awk -F\'title="\' \'{print $2}\' |awk -F\'"\' \'{print $1}\'  '], stdout=PIPE, stderr=PIPE, shell=True).stdout).replace("b'", "")).split("\\n")[0]
    
    eo_href = (str(run([f' ''curl --proxy1.0 \'http://sysproxy.wal-mart.com:8080\' -L -H \'Token: mde170cede5311120e6b6694d736356c1\' \'https://www.irs.gov/charities-non-profits/exempt-organizations-business-master-file-extract-eo-bmf\' |grep "'+region+'".*pup_linkit_media  |awk -F\'g>\' \'{print $1}\' |awk -F\'</\' \'{print $1}\'  |awk -F\'href="\' \'{print $2}\' |awk -F\'"\' \'{print $1}\'  '], stdout=PIPE, stderr=PIPE, shell=True).stdout).replace("b'", "")).split("\\n")[0]
    
    eo_uuid = (str(run([f' ''curl --proxy1.0 \'http://sysproxy.wal-mart.com:8080\' -L -H \'Token: mde170cede5311120e6b6694d736356c1\' \'https://www.irs.gov/charities-non-profits/exempt-organizations-business-master-file-extract-eo-bmf\' |grep pup_linkit_media  |awk -F\'g>\' \'{print $1}\' |awk -F\'</\' \'{print $1}\'  |awk -F\'data-entity-uuid="\' \'{print $2}\' |awk -F\'"\' \'{print $1}\'  '], stdout=PIPE, stderr=PIPE, shell=True).stdout).replace("b'", "")).split("\\n")[0]
    
    print('\n\neo_file =\t{}\neo_href =\t{}\neo_uuid =\t{}'.format(eo_file,eo_href,eo_uuid))
    eo_bmf_dict[eo_href]=eo_file
    eo = eo_file.replace('.csv', '')
    
    str(run([f'curl --proxy1.0 \'http://sysproxy.wal-mart.com:8080\' -L -H \'Token: mde170cede5311120e6b6694d736356c1\' \'{eo_href}\' |sed -e "s/% //g" -e "s/%//g" > {HOME}/{eo}_bmf_extract_{post_date}.csv '], stdout=PIPE, stderr=PIPE, shell=True).stdout)
    
    connector.upload(name='INKIRU_CLOUD', container='inkiru-ds-prodjobs', source_path=f'{HOME}/{eo}_bmf_extract_{post_date}.csv', target_path=f'NONPROFITS/{eo}_bmf_extract_{post_date}.csv')
    

    eo_bmf_uri = f'gs://inkiru-ds-prodjobs/NONPROFITS/{eo}_bmf_extract_{post_date}.csv'
    eo_bmf_tab = project+f'.EO_BMF_EXTRACT.{eo}'
    load_job = bqclient.load_table_from_uri(eo_bmf_uri, eo_bmf_tab, job_config=eo_bmf_fig)

    load_job.result()
    
    rpt_sum = dataset.load(name='bq2', query=f"select dataset_id, table_id, ROUND(size_bytes/POW(10,9),2) AS size_gb, TIMESTAMP_MILLIS(last_modified_time) AS last_modified_time, row_count from EO_BMF_EXTRACT.__TABLES__ where table_id ='{eo}'; ", sep=';')
    print(rpt_sum)
    
    with open(f'{HOME}eo_bmf_region.csv', 'a') as msg_2:
        msg_2.writelines('{}\n\n'.format(rpt_sum))


with open('/home/jupyter/eo_bmf_region.csv', 'r') as region:
    region = region.read()
    run([f'rm {HOME}eo_bmf_region.csv'], shell=True, encoding='utf-8')

nonprofit = f"""
CREATE OR REPLACE TABLE `wmt-inkiru-ds.EO_BMF_EXTRACT.nonprofit_full_list` (
  tax_id STRING NOT NULL,
  organization_name STRING NOT NULL,
  recipient_name STRING,
  street STRING,
  city STRING,
  state STRING,
  country STRING NOT NULL,
  foundation_code STRING NOT NULL,
  updated_ts STRING NOT NULL,
  PRIMARY KEY (tax_id) NOT ENFORCED
) AS
SELECT DISTINCT
  TO_HEX(SHA256(EIN)) AS tax_id,
  Name AS organization_name,
  ICO AS recipient_name,
  STREET AS street,
  CASE WHEN STATE IS NULL THEN NULL ELSE city END AS city,
  STATE AS state,
  CASE WHEN STATE IS NULL THEN city ELSE 'UNITED STATES' END AS country,
  FOUNDATION AS foundation_code,
  FORMAT_TIMESTAMP('%Y-%m-%d %H:%M:%S', CURRENT_TIMESTAMP()) AS updated_ts
FROM (
  SELECT EIN, Name, ICO, STREET, CITY, STATE, FOUNDATION FROM `wmt-inkiru-ds.EO_BMF_EXTRACT.eo1`
  UNION ALL
  SELECT EIN, Name, ICO, STREET, CITY, STATE, FOUNDATION FROM `wmt-inkiru-ds.EO_BMF_EXTRACT.eo2`
  UNION ALL
  SELECT EIN, Name, ICO, STREET, CITY, STATE, FOUNDATION FROM `wmt-inkiru-ds.EO_BMF_EXTRACT.eo3`
  UNION ALL
  SELECT EIN, Name, ICO, STREET, CITY, STATE, FOUNDATION FROM `wmt-inkiru-ds.EO_BMF_EXTRACT.eo4`
);
"""

nonprof_post_date = nonprofit.replace('nonprofit_full_list', f'nonprofit_full_list_{post_date}')

connector.execute(name='bq2', statement=nonprofit, sep=';')
connector.execute(name='bq2', statement=nonprof_post_date, sep=';')

df_eo = dataset.load(name='bq2', query='select * from EO_BMF_EXTRACT.nonprofit_full_list')
df_eo.to_json(f'{HOME}IRS_exempt_organization_list_{list_date}.json', orient = 'records', lines= 'true', index = 'false',date_format = "iso")
run([f'touch {HOME}IRS_exempt_organization_list_{list_date}.json.gz ; rm {HOME}IRS_exempt_organization_list_{list_date}.json.gz ; gzip {HOME}IRS_exempt_organization_list_{list_date}.json'], shell=True, encoding='utf-8')
run([f'chmod 600 {HOME}id_rsa-sftpgec'], shell=True, encoding='utf-8')
run([f'sftp -oIdentityfile={HOME}id_rsa-sftpgec -oHostKeyAlgorithms=+ssh-rsa -oPubkeyAcceptedAlgorithms=+ssh-rsa -oPort=2222 gec.com@file-exchange-dr.walmart.com <<EOT \nmkdir /inbox/cds_data_import/IRS_exempt_organization_list/ \ncd /inbox/cds_data_import/IRS_exempt_organization_list/ \nput {HOME}IRS_exempt_organization_list_{list_date}.json.gz \nEOT'], shell=True, encoding='utf-8')

file_chk = ((str(run([f'sftp -oIdentityfile={HOME}id_rsa-sftpgec -oHostKeyAlgorithms=+ssh-rsa -oPubkeyAcceptedAlgorithms=+ssh-rsa -oPort=2222 gec.com@file-exchange-dr.walmart.com <<EOT \nls -l /inbox/cds_data_import/IRS_exempt_organization_list/IRS_exempt_organization_list_{list_date}.json.gz \nEOT'], capture_output=True, shell=True, encoding='utf-8')).split('?')[1]).split('\\')[0]).replace('0      ','')

final_cnt = str(dataset.load(name='bq2', query='select count(*) from EO_BMF_EXTRACT.nonprofit_full_list;', sep=';').to_csv).replace('>','').split()[-1]

with open('/home/jupyter/eo_bmf_report.csv', 'a') as msg_1:
    msg_1.writelines('final count = {}'.format(final_cnt))
    msg_1.writelines('\n\n\n\r[ eo1 eo2 eo3 eo4]')
    msg_1.writelines('\n\n\n')
    msg_1.writelines('{}'.format(region))
    

for eo in ['eo1', 'eo2', 'eo3', 'eo4']:
    bqclient.delete_table(project+f'.EO_BMF_EXTRACT.{eo}')
    
    

with open('/home/jupyter/eo_bmf_report.csv', 'r') as rpt:
    rpt = rpt.read()

msg_s = (rpt)

wkflo = 'https://console.dx.walmart.com/element/workzone/projects/10878/workflows/41736'

msg = MIMEMultipart()
msg['From']='GEC ADMIN<svcgecadmin@walmart.com>'
msg['To']='Yutong.Yang@walmart.com,Siyuan.Zhou0@walmart.com,INK-DS-OPS@email.wal-mart.com'
msg['Cc']='Brad.Leonard0@walmart.com'
msg['Subject']="IRS Exempt Organization BFM Extraction completed for "+upd_post_date+" "
msg.attach(MIMEText (str(msg_s)+'\n\nCDS upload completed  '+str(file_chk)+'\n\n\n'+str(wkflo)))
s = smtplib.SMTP(host='phont300cdcus1.homeoffice.Wal-Mart.com', port=25)
s.send_message(msg)
del msg



